In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import os
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, BooleanType
from pyspark.sql.functions import collect_list, udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import first
import pyspark
from typing import List, Dict, Tuple, Set, Optional, Callable
from pyspark.sql import DataFrame, Column, Row

# Сначала подготовим датасет с последовательностями конверсий

In [3]:
def clean_data_actions(data_logs: pyspark.sql.DataFrame, action: str) -> pyspark.sql.DataFrame:
    """
    Обрабатывает датасет, оставляя только данные о нужном действии и добавляя необходимые столбцы.

    Args:
        data_logs (pyspark.sql.DataFrame): Входной датасет с логами действий пользователей. Должен содержать столбцы:
            - action: Действие пользователя (например, 'conversion').
            - customer_user_id: Уникальный идентификатор пользователя.
            - customer_id: Уникальный идентификатор товара.
            - timestamp: Временная метка действия.
            - revenue: Доход, полученный от действия.
        action (str): Названия действия для фильтрации датасета

    Returns:
        pyspark.sql.DataFrame: Очищенный датасет со следующими столбцами:
            - user_id: Уникальный идентификатор пользователя.
            - item_id: Уникальный идентификатор товара или услуги.
            - timestamp: Временная метка конверсии.
            - revenue: Доход, полученный от конверсии.
    """
    # Фильтруем данные, оставляя только строки с действием 'conversion'
    data_actions = data_logs.filter(data_logs.action == action)

    # Оставляем только нужные столбцы: customer_user_id, customer_id, timestamp, revenue
    data_actions_filtered = data_actions.select('customer_user_id', 'customer_id', 'timestamp', 'revenue')

    # Сортируем данные по customer_user_id и timestamp
    data_actions = data_actions_filtered.orderBy(['customer_user_id', 'timestamp'])

    # Переименовываем столбцы для удобства
    data_actions = data_actions.withColumnRenamed('customer_user_id', 'user_id') \
                      .withColumnRenamed('customer_id', 'item_id')

    # Проверяем датасет на пропущенные значения
    if check_nan_values(data_actions) > 0:
      data_actions = data_actions.dropna()

    return data_actions


def check_nan_values(data: pyspark.sql.DataFrame) -> int:
    """
    Проверяет пропущенные значения (NULL и NaN) в датасете.

    Args:
        data (pyspark.sql.DataFrame): Входной датасет.

    Returns:
        int: число строчек с NULL
    """
    # Подсчёт строк до удаления пропущенных значений
    initial_count = data.count()

    # Удаляем строки с пропущенными значениями
    data_cleaned = data.dropna()

    # Подсчёт строк после удаления пропущенных значений
    final_count = data_cleaned.count()

    # Вычисляем количество удалённых строк
    removed_count = initial_count - final_count
    print(f"Найдено строк c пропущенных значений: {removed_count}")

    # Проверяем, в каких колонках есть пропущенные значения
    for column in data.columns:
        # Получаем тип колонки
        column_type = str(data.schema[column].dataType)

        # Проверяем только NULL для нечисловых колонок
        if column_type not in ["DoubleType", "FloatType"]:
            na_count = data.filter(col(column).isNull()).count()
            if na_count > 0:
                print(f"Колонка '{column}' содержит {na_count} пропущенных значений (NULL).")
        else:
            # Для числовых колонок проверяем и NULL, и NaN
            na_count = data.filter(col(column).isNull() | col(column).isNaN).count()
            if na_count > 0:
                print(f"Колонка '{column}' содержит {na_count} пропущенных значений (NULL или NaN).")

    return removed_count

In [5]:
def actions_to_sasrec_format(data_actions_cleaned: pyspark.sql.DataFrame) -> pyspark.sql.DataFrame:
    """
    Преобразует датасет с действиями юзеров в формат, подходящий для модели SASRec.
    А именно указывает соотвествующий тип данных в датасете и отбрасывает не нужные столбцы

    Args:
        data_actions_cleaned (pyspark.sql.DataFrame): Датасет (полученный функцией clean_data_actions от изначального датасета needed_beh_logs)
        только с нужными действиями покупателей. Должен содержать столбцы:
            - user_id: Уникальный идентификатор пользователя.
            - item_id: Уникальный идентификатор товара или услуги.
            - timestamp: Временная метка действия.

    Returns:
        pyspark.sql.DataFrame: Преобразованный датасет со следующими столбцами:
            - user_id: Уникальный идентификатор пользователя (тип int).
            - item_id: Уникальный идентификатор товара или услуги (тип int).
            - datetime: Временная метка действия в формате timestamp.
            - weight: Вес действия (всегда 1, тип int).
    """
    # Преобразуем timestamp в строку с датой и временем и добавляем колонку weight
    data_actions_processed = data_actions_cleaned.withColumn(
        "datetime", F.date_format("timestamp", "yyyy-MM-dd HH:mm:ss")  # Преобразуем timestamp в строку
    ).withColumn(
        "weight", F.lit(1)  # Добавляем колонку weight со значением 1
    ).select("user_id", "item_id", "datetime", "weight")  # Оставляем нужные колонки

    # Приводим типы данных к нужным форматам
    data_actions_processed = data_actions_processed \
        .withColumn("user_id", col("user_id").cast("int")) \
        .withColumn("item_id", col("item_id").cast("int")) \
        .withColumn("datetime", col("datetime").cast("timestamp")) \
        .withColumn("weight", col("weight").cast("int"))

    # Проверяем датасет на пропущенные значения
    check_nan_values(data_actions_processed)

    # Проверяем датасет на пропущенные значения
    if check_nan_values(data_actions_processed) > 0:
      data_actions_processed = data_actions_processed.dropna()

    return data_actions_processed

# Теперь подготовим датасет с описанием товаров

In [6]:
import json
from pyspark.sql import DataFrame
import pyspark

def print_metadata(df: pyspark.sql.DataFrame):
    """Выводит основные метаданные и отдельные значения facets из JSON в колонке metadata_json."""
    first_row = df.select("metadata_json").first()
    if not first_row:
        print("Датасет пуст.")
        return

    metadata = json.loads(first_row["metadata_json"])

    # Выводим основные метаданные
    print("Основные метаданные:")
    for key, value in metadata.items():
        if key != "facets":
            print(f"{key}: {value}")

    # Обрабатываем ключ facets отдельно
    if "facets" in metadata:
        print("\nFacets:")
        facets = json.loads(metadata["facets"]) if isinstance(metadata["facets"], str) else metadata["facets"]
        for key, value in facets.items():
            print(f"{key}: {value}")

In [8]:
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
import pyspark
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
from typing import List, Optional

def parse_metadata_json(data_items: pyspark.sql.DataFrame, metadata: List[str], facets: Optional[List[str]] = None) -> pyspark.sql.DataFrame:
    """Извлекает данные из metadata_json и добавляет их как отдельные колонки."""
    metadata_schema_fields = [
        StructField(field, ArrayType(StringType()), True) if field == "group_ids" else StructField(field, StringType(), True)
        for field in metadata
    ]

    if facets:
        metadata_schema_fields.append(
            StructField("facets", StructType([StructField(facet, ArrayType(StringType()), True) for facet in facets]), True)
        )

    metadata_schema = StructType(metadata_schema_fields)

    data_items_parsed = data_items.withColumn(
        "metadata_parsed", F.from_json(F.col("metadata_json"), metadata_schema)
    )

    selected_columns = ["id", "customer_id", "name", "name_lower"] + [f"metadata_parsed.{field}" for field in metadata]
    if facets:
        selected_columns.append("metadata_parsed.facets")

    return data_items_parsed.select(*selected_columns)

def find_intersection(data_items: pyspark.sql.DataFrame, group_col: str, value_col: str) -> pyspark.sql.DataFrame:
    """Находит пересечение значений в колонке value_col для каждой группы, определенной group_col.

    Args:
        data_items (DataFrame): Входной датасет.
        group_col (str): Название колонки, по которой будет группировка (например, customer_id).
        value_col (str): Название колонки, для которой ищется пересечение значений. (например, group_ids)

    Returns:
        DataFrame: Датасет с group_col и пересечением значений из value_col.
    """
    exploded_df = data_items.select(group_col, F.explode(value_col).alias("value"))

    grouped_df = exploded_df.groupBy(group_col, "value").agg(F.count("*").alias("count"))
    total_lists_per_group = data_items.groupBy(group_col).agg(F.count("*").alias("total_lists"))

    joined_df = grouped_df.join(total_lists_per_group, on=group_col, how="left")

    return joined_df.filter(F.col("count") == F.col("total_lists")) \
        .groupBy(group_col) \
        .agg(F.collect_list("value").alias(f"{value_col}_intersect"))

def aggregate_data_items(data_items: pyspark.sql.DataFrame, metadata: List[str], facets: Optional[List[str]] = None) -> pyspark.sql.DataFrame:
    """Агрегирует данные по customer_id, беря первые значения одинаковых колонок.
       В тех колонках, где внутри одного customer_id значения не меняются, берем первые из этих значений для опредления признака данного customer_id
    """
    same_columns = ['name', 'name_lower'] + metadata
    if facets:
        same_columns.append("facets")

    return data_items.groupBy("customer_id").agg(
        *[F.first(col_name).alias(col_name) for col_name in same_columns if col_name != "customer_id"]
    )

def clean_data_items(data_items: pyspark.sql.DataFrame, metadata: List[str], facets: Optional[List[str]] = None) -> pyspark.sql.DataFrame:
    """Убирает лишние колонки, разворачивает нужные метаданные и facets в новые колонки, а также агрегирует признаки внутри одних customer_id"""
    data_items_filtered = data_items.select("id", "customer_id", "name", "name_lower", "metadata_json")
    data_items_parsed = parse_metadata_json(data_items_filtered, metadata, facets)

    aggregated_same = aggregate_data_items(data_items_parsed, metadata, facets)

    # Находим внутри каждого customer_id перечение group_ids
    if "group_ids" in metadata:
        intersected_df = find_intersection(data_items_parsed, "customer_id", "group_ids")
        final_df = aggregated_same.join(intersected_df.select("customer_id", "group_ids_intersect"), on="customer_id", how="left")
        final_df = final_df.drop("group_ids")
    else:
        final_df = aggregated_same

    # Извлекаем отдельные атрибуты из facets, если facets передан, и вставляем их в датасет в качестве новых колонок
    if facets:
        for facet in facets:
            final_df = final_df.withColumn(facet.replace(" ", "_"), F.col(f"facets.`{facet}`"))
        final_df = final_df.drop("facets")

    return final_df

In [9]:
from pyspark.sql import DataFrame
import pyspark
from typing import List

def items_to_sasrec_format(
    data_items_cleaned: pyspark.sql.DataFrame, # результат функции clean_data_items от изначального датасета data_set_items
    features: List[str],
    features_names_out: List[str]
) -> pyspark.sql.DataFrame:
    """Преобразует данные о товарах в формат SASRec.

    Args:
        data_items_cleaned (DataFrame): Исходный DataFrame с товарами.
        features (List[str]): Список названий колонок с фичами.
        features_names_out (List[str]): Список имен фичей для выхода.

    Returns:
        DataFrame: Преобразованный DataFrame с фичами в формате SASRec.
    """
    df = data_items_cleaned.select(*(['customer_id'] + features))
    df = df.withColumn('customer_id', F.col('customer_id').cast('int'))
    dfs_feature = []

    for i in range(len(features)):
        feature = df.select(
            "customer_id",
            F.explode(features[i]).alias("value")
        ).withColumn("feature", F.lit(features_names_out[i])).withColumnRenamed("customer_id", "id")
        dfs_feature.append(feature)

    data_items_processed = dfs_feature[0]
    for i in range(1, len(features)):
        data_items_processed = data_items_processed.union(dfs_feature[i])

    return data_items_processed


# Прочие функции


In [10]:
def save_dataset_parquet(dataset: pyspark.sql.DataFrame, folder_path: str, file_name: str) -> None:
    """
    Сохраняет датасет в формате Parquet по указанному пути.

    Args:
        dataset (pyspark.sql.DataFrame): Датасет, который нужно сохранить.
        folder_path (str): Путь к папке, куда будет сохранён файл.
        file_name (str): Имя файла для сохранения (без расширения).

    Returns:
        None: Функция не возвращает значение, но сохраняет датасет на диск.
    """
    # Проверяем, существует ли папка, если нет - создаем
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Папка {folder_path} успешно создана!")
    else:
        print(f"Папка {folder_path} уже существует.")

    # Формируем полный путь к файлу
    file_path = os.path.join(folder_path, file_name)

    # Сохранение в формате Parquet
    dataset.write.parquet(file_path, mode="overwrite")
    print(f"Датасет сохранён по пути {file_path}.")